### Analysis of the data and information in the .csv files

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

import datetime

In [ ]:
iD= ''
Date='2019-03-18'
WeekDay= datetime.datetime.strptime(Date, '%Y-%m-%d').strftime('%A')
df_ID_2019=pd.read_csv('d/'+iD+'.csv')
df_metadata = pd.read_csv('a.csv') # To see more information about the identifier, such as the number of sensors
df_ID_2019[['n']].head()

In [ ]:
#Columns in the .csv file
df_ID_2019.columns.values.tolist()

In [ ]:
#  information
No_RS = ((df_metadata[df_metadata['Identifier']==iD])['Number of Remote Sensors']).iloc[0]
print('Number of Sensors %i' %(No_RS+1))
print('Number of Remote Sensors %i' %(No_RS))
No_Occup = ((df_metadata[df_metadata['Identifier']==iD])['Number of Occupants']).iloc[0]
print('Number of Occupants %i' %(No_Occup))
Country1 = ((df_metadata[df_metadata['Identifier']==iD])['Country']).iloc[0]
print('Country %s' %(Country1))
ProvinceState1 = ((df_metadata[df_metadata['Identifier']==iD])['ProvinceState']).iloc[0]
print('ProvinceState %s' %(ProvinceState1))

In [ ]:
# " % of RS in 1 in a day, Identifier with 6 sensors and 2 occupants, 2019-03-18(Monday)"
df_in_2019_day = df_ID_2019.loc[ df_ID_2019.DateTime.str.contains(Date).to_numpy(),:]
df_in_2019_day=df_in_2019_day.reset_index(drop=True)

df_time_activation=pd.DataFrame()
df_time_activation['DateTime'] = df_in_2019_day['DateTime'].astype('datetime64[ns]') 
df_time_activation['Time'] = df_time_activation['DateTime'].dt.time 
df_time_activation['Date'] = df_time_activation['DateTime'].dt.date 
df_time_activation['activation'] = df_in_2019_day.loc[:, df_in_2019_day.columns.str.contains('_Motion')].sum(axis=1)/(No_RS+1)

xtick= np.arange(start=0, stop=len(df_time_activation), step=4)
ax =df_time_activation[['Time','activation']].plot(kind='line',figsize=(18, 6), xlim=(0,len(df_time_activation)), xticks= xtick )
ax.set_xticklabels(df_time_activation.Time[xtick],rotation=90)
ax.set_title(" %% of RS in 1 in a day, Identifier with %i sensors and %i occupants, %s(%s)" %(No_RS+1,No_Occup,Date,WeekDay))
ax.set_xlabel('Time [Hs]')
ax.set_ylabel('% of RS in 1')

In [ ]:
# " Activation of RS in a day, Identifier with 6 sensors and 2 occupants, 2019-03-18(Monday)"
df_in_2019_day2 = df_ID_2019.loc[ df_ID_2019.DateTime.str.contains(Date).to_numpy(),:]
df_in_2019_day2=df_in_2019_day2.reset_index(drop=True)

df_time_activation2 = df_in_2019_day2.loc[:, df_in_2019_day.columns.str.contains('_Motion')]
df_time_activation2['DateTime'] = df_in_2019_day2['DateTime'].astype('datetime64[ns]') 
df_time_activation2['Time'] = df_time_activation2['DateTime'].dt.time 
df_time_activation2['Date'] = df_time_activation2['DateTime'].dt.date 

drop2=['']
df_time_activation_graph2 = df_time_activation2.set_index(df_time_activation2['Time']).drop(drop2,axis=1)
fig2,axx2=plt.subplots(figsize=(18,6))

sns.heatmap(df_time_activation_graph2.T,cmap='Blues',ax=axx2)
axx2.set_title('Activation of RS in a day, Identifier with %i sensors and %i occupants, %s(%s)' %(No_RS+1,No_Occup,Date,WeekDay))
axx2.set_xlabel('Time [Hs]')

In [32]:
# Graph of the average activation of each sensor on a day of the week during the specified period
# The start day is included, the end day is not
def activationDayEachRS(weekday,start_date,end_date):
    df_time_activation3_1 = df_ID_2019.loc[:, df_in_2019_day.columns.str.contains('_Motion')]
    # Time period selection
    df_time_activation3_1['DateTime'] = df_ID_2019['DateTime'].astype('datetime64[ns]')
    df_time_activation3_1['Date'] = df_time_activation3_1['DateTime'].dt.date 
    df_time_activation3=df_time_activation3_1[(df_time_activation3_1['Date'] >= start_date) & (df_time_activation3_1['Date'] < end_date)]
    #Selection of the day of the week
    df_time_activation3['dayofweek']=df_time_activation3['DateTime'].dt.day_name()
    df_time_activation3['Time'] = df_time_activation3['DateTime'].dt.time
    df_in_2019_day3 = df_time_activation3.loc[ df_time_activation3.dayofweek.str.contains(weekday).to_numpy(),:]
    df_in_2019_day3=df_in_2019_day3.reset_index(drop=True)

    drop3=['DateTime','dayofweek' ,'Date', 'Remote_Sensor_6_Motion','Remote_Sensor_7_Motion','Remote_Sensor_8_Motion','Remote_Sensor_9_Motion','Remote_Sensor_10_Motion']
    df_time_activation_graph3 = (df_in_2019_day3.drop(drop3,axis=1)).groupby('Time').mean()
 
    fig2,axx_3=plt.subplots(figsize=(18,6))
    xtick3= np.arange(start=0, stop=(df_time_activation_graph3.T.shape)[1])
    sns.heatmap(df_time_activation_graph3.T,cmap='viridis',ax=axx_3)
    axx_3.set_title('')
    axx_3.set_xlabel('Time [Hs]')

In [ ]:
activationDayEachRS('Monday',datetime.date(2019, 4, 1),datetime.date(2019, 5, 1))

In [ ]:
%matplotlib

In [12]:
# -------- Function: Activation percentage on one day of the week, four different dates within a month
def Activation_percentage_weekday(Date_month,weekday):

    df_in_2019_month = df_ID_2019.loc[ df_ID_2019.DateTime.str.contains(Date_month).to_numpy(),:]
    df_in_2019_month=df_in_2019_month.reset_index(drop=True)

    df_week_activation=pd.DataFrame()
    df_week_activation['activation'] = df_in_2019_month.loc[:, df_in_2019_month.columns.str.contains('_Motion')].sum(axis=1)/(No_RS+1)
    df_week_activation['DateTime'] = df_in_2019_month['DateTime'].astype('datetime64[ns]') 
    df_week_activation['dayofweek']=df_week_activation['DateTime'].dt.day_name()
    df_week_activation['Time'] = df_week_activation['DateTime'].dt.time 
    df_week_activation['Date'] = df_week_activation['DateTime'].dt.date 

    df_week_activation = df_week_activation.loc[df_week_activation.dayofweek.str.contains(weekday).to_numpy(),:]
    df_week_activation=df_week_activation.reset_index(drop=True)
    drop=['DateTime']
    df_week_activation_graph = df_week_activation.drop(drop,axis=1)
    group_week_activation_graph=df_week_activation_graph.groupby(['Date','Time']).mean()

    pp=pd.DataFrame(group_week_activation_graph.to_records())
    r_week_activation=pp.pivot_table(index='Date', columns='Time', values='activation',fill_value=0)
    r_week_activation=r_week_activation.head(4)
    # ----- Plot ---------
    fig = plt.figure(figsize=(18,6))
    ax1 =  Axes3D(fig)

    xtick=np.arange(start=0, stop=len(r_week_activation.T.index), step=6)
    xaxis= np.arange(start=0, stop=len(r_week_activation.T.index))
    ax1.set_xticks(xtick)
    ax1.set_xticklabels(r_week_activation.T.index[xtick],rotation=90)

    ytick=np.arange(start=0, stop=len(r_week_activation.index), step=1)
    ax1.set_yticks(ytick)
    ax1.set_yticklabels(r_week_activation.index[ytick],rotation=0)

    ax1.set_title("Activation percentage on 4 %ss of %s" %(weekday,Date_month))
    ax1.set_xlabel('Time') 
    ax1.set_zlabel('Activation percentage')
    ax1.set_ylabel('Date')

    for i in [0,1,2,3]:
        ax1.plot(xs=xaxis,ys=r_week_activation.T.iloc[:, i], zs=i, zdir='y')

In [ ]:
# Activation percentage on one day of the week, four different dates within a month
Date_month='2019-03'
weekday=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']
Activation_percentage_weekday(Date_month,weekday[0])
Activation_percentage_weekday(Date_month,weekday[1])
Activation_percentage_weekday(Date_month,weekday[2])
Activation_percentage_weekday(Date_month,weekday[3])
Activation_percentage_weekday(Date_month,weekday[4])
Activation_percentage_weekday(Date_month,weekday[5])
Activation_percentage_weekday(Date_month,weekday[6])

In [10]:
#%matplotlib

Using matplotlib backend: Qt5Agg


In [7]:
# Function to plot the activation percentage of RS for each day of the week during a period
def activation_week_day(weekday,start_date,end_date):

    df_alldays_week = pd.DataFrame()
    for i_weekday in weekday:

        df_datetime= df_ID_2019.copy()
        df_datetime['DateTime'] = df_datetime['DateTime'].astype('datetime64[ns]') 
        df_datetime['Date'] = df_datetime['DateTime'].dt.date
        df_in_2019_month=df_ID_2019[(df_datetime['Date'] >= start_date) & (df_datetime['Date'] < end_date)]
        df_in_2019_month=df_in_2019_month.reset_index(drop=True)

        df_week_activation=pd.DataFrame()
        df_week_activation['activation'] = df_in_2019_month.loc[:, df_in_2019_month.columns.str.contains('_Motion')].sum(axis=1)/(No_RS+1)
        df_week_activation['DateTime'] = df_in_2019_month['DateTime'].astype('datetime64[ns]') 
        df_week_activation['dayofweek']=df_week_activation['DateTime'].dt.day_name()
        df_week_activation['Time'] = df_week_activation['DateTime'].dt.time 
        df_week_activation['Date'] = df_week_activation['DateTime'].dt.date 

        df_week_activation = df_week_activation.loc[df_week_activation.dayofweek.str.contains(i_weekday).to_numpy(),:]
        df_week_activation=df_week_activation.reset_index(drop=True)
        drop=['DateTime']
        df_week_activation_graph = df_week_activation.drop(drop,axis=1)
        group_week_activation_graph=df_week_activation_graph.groupby(['Date','Time']).mean()

        pp=pd.DataFrame(group_week_activation_graph.to_records())
        r_week_activation=pp.pivot_table(index='Date', columns='Time', values='activation',fill_value=0)
        r_week_activation=pd.DataFrame(r_week_activation.mean(axis = 0))
        r_week_activation['dayofweek']=i_weekday
        df_alldays_week = pd.concat([df_alldays_week,r_week_activation])
    #--------------Heatmap---------------------------------------------

    group_week_activation_graph=df_alldays_week.groupby(['dayofweek','Time']).mean()
    pp=pd.DataFrame(group_week_activation_graph.to_records())
    r_week_activation_2=pp.pivot_table(index='dayofweek', columns='Time', values='0',fill_value=0)

    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']
    fig2,axx3=plt.subplots(figsize=(18,8))
    sns.heatmap(r_week_activation_2.reindex(days), cmap='viridis',ax=axx3,vmax=0.44)
    axx3.set_title('Activation percentage for each day of the week from %s to %s' %(start_date,end_date))
    axx3.set_xlabel('Time [Hs]')

    #-----------------------------------------------------------
    group_alldays_week=df_alldays_week.groupby('dayofweek',sort=False)

    fig = plt.figure()
    ax_i_weekday = Axes3D(fig)
    
    xtick=np.arange(start=0, stop=288, step=12)
    xaxis= np.arange(start=0, stop=288)
    ax_i_weekday.set_xticks(xtick)
    ax_i_weekday.set_xticklabels(r_week_activation.index[xtick],rotation=90)

    ax_i_weekday.set_title('Activation percentage for each day of the week from %s to %s' %(start_date,end_date))
    ax_i_weekday.set_xlabel('Time') 
    ax_i_weekday.set_zlabel('Activation percentage')
    ax_i_weekday.set_ylabel('dayofweek')

    a=[]
    i=0
    for key, group_df in group_alldays_week:
        # `key` contains the name of the grouped element 
        # `group_df` is a normal dataframe
        ax_i_weekday.plot(xs=xaxis,ys=group_df[0], zs=i, zdir='y')
        a.append(key)
        i=i+1

    ytick=np.arange(start=0, stop=len(group_alldays_week), step=1)
    ax_i_weekday.set_yticks(ytick)
    ax_i_weekday.set_yticklabels(a,rotation=0)


In [ ]:
#---activation percentage of RS for each day of the week during a period
weekday=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']
start_date = datetime.date(2019, 3, 1)
end_date = datetime.date(2019, 3, 8)
activation_week_day(weekday,start_date,end_date)

In [34]:
#--- Activation percentage for the week and the weekend during a period

def activation_week_weekend_period(weekday,start_date,end_date):

    df_alldays_week = pd.DataFrame()
    for i_weekday in weekday:
    
        df_datetime= df_ID_2019.copy()
        df_datetime['DateTime'] = df_datetime['DateTime'].astype('datetime64[ns]') 
        df_datetime['Date'] = df_datetime['DateTime'].dt.date
        df_in_2019_month=df_ID_2019[(df_datetime['Date'] >= start_date) & (df_datetime['Date'] < end_date)]
        df_in_2019_month=df_in_2019_month.reset_index(drop=True)

        df_week_activation=pd.DataFrame()
        df_week_activation['activation'] = df_in_2019_month.loc[:, df_in_2019_month.columns.str.contains('_Motion')].sum(axis=1)/(No_RS+1)
        df_week_activation['DateTime'] = df_in_2019_month['DateTime'].astype('datetime64[ns]') 
        df_week_activation['dayofweek']=df_week_activation['DateTime'].dt.day_name()
        df_week_activation['Time'] = df_week_activation['DateTime'].dt.time 
        df_week_activation['Date'] = df_week_activation['DateTime'].dt.date 

        df_week_activation = df_week_activation.loc[df_week_activation.dayofweek.str.contains(i_weekday).to_numpy(),:]
        df_week_activation=df_week_activation.reset_index(drop=True)
        drop=['DateTime']
        df_week_activation_graph = df_week_activation.drop(drop,axis=1)
        group_week_activation_graph=df_week_activation_graph.groupby(['Date','Time']).mean()

        pp=pd.DataFrame(group_week_activation_graph.to_records())
        r_week_activation=pp.pivot_table(index='Date', columns='Time', values='activation',fill_value=0)
        r_week_activation=pd.DataFrame(r_week_activation.mean(axis = 0))
        if i_weekday in weekday[0:5]:
            r_week_activation['dayofweek']='week'
        else:
            r_week_activation['dayofweek']='weekend'

        df_alldays_week = pd.concat([df_alldays_week,r_week_activation])

  #--------------Heatmap---------------------------------------------
    group_week_activation_graph=df_alldays_week.groupby(['dayofweek','Time']).mean()
    pp=pd.DataFrame(group_week_activation_graph.to_records())
    r_week_activation_2=pp.pivot_table(index='dayofweek', columns='Time', values='0',fill_value=0)

    days = ['week', 'weekend']
    fig2,axx3=plt.subplots(figsize=(10,8))
    sns.heatmap(r_week_activation_2.reindex(days), cmap='viridis',ax=axx3)
    axx3.set_title('Activation percentage for each day of the week from %s to %s' %(start_date,end_date))
    axx3.set_xlabel('Time [Hs]')
   
    #-----------------------------------------------------------
    df_alldays_week=df_alldays_week.reset_index()
    group_alldays_week=df_alldays_week.groupby(['dayofweek','Time']).mean()
    group_alldays_week_graph = pd.DataFrame(group_alldays_week.to_records())
    group_graph=group_alldays_week_graph.groupby('dayofweek')

    fig = plt.figure()
    ax_i_weekend = Axes3D(fig)

    xtick=np.arange(start=0, stop=288, step=12)
    xaxis= np.arange(start=0, stop=288)
    ax_i_weekend.set_xticks(xtick)
    ax_i_weekend.set_xticklabels(r_week_activation.index[xtick],rotation=90)

    ax_i_weekend.set_title('Activation percentage for each day of the week from %s to %s' %(start_date,end_date))
    ax_i_weekend.set_xlabel('Time') 
    ax_i_weekend.set_zlabel('Activation percentage')
    ax_i_weekend.set_ylabel('dayofweek')

    a=[]
    i=0
    for key, group_df in group_graph:
        # `key` contains the name of the grouped element 
        # `group_df` is a normal dataframe
        ax_i_weekend.plot(xs=xaxis,ys=group_df['0'], zs=i, zdir='y')
        a.append(key)
        i=i+0.2

    ytick=np.arange(start=0, stop=1, step=0.2)
    yaxis= np.arange(start=0, stop=1,step=0.2)
    ax_i_weekend.set_yticks(ytick)
    ax_i_weekend.set_yticklabels(a,rotation=0)

In [ ]:
weekday=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']
start_date = datetime.date(2019, 3, 21)
end_date = datetime.date(2019, 6, 21)
activation_week_weekend_period(weekday,start_date,end_date)